In [ ]:
import numpy as np
from glob import glob
import itertools
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=20)


import matplotlib.pyplot as plt
import seaborn as sns
from src.utils.parse_config import read_config_file

from os.path import join
import os
import pandas as pd

In [ ]:
params_f = "/data/Mito_Trace/parameters/pipeline/pipeline.yaml"

# prefix = ""
# cfg_outdir = ""
# outdir = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/comparisons"
# all_files = "/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_10/concat/cells_meta.tsv,/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/knn/kparam_3/concat/cells_meta.tsv,/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_10/concat/cells_meta.tsv,/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/cells_meta.tsv,/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/cells_meta.tsv,/data/Mito_Trace/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/vireo/nclones20/cells_meta.tsv"

prefix = "CHIP_b1/MTBlacklist_A2"
cfg_outdir = "/data/Mito_Trace/output"
outdir = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/comparisons"
all_files = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/cells_meta.tsv" 

#names = ""
#cfg_f = "/data/Mito_Trace/parameters/pipeline/cosmo_server/jan21_2021.yaml"

exclude = [{"variants":"simple", 
           "method": "knn"}]

In [ ]:
all_files = all_files.split(",")

In [ ]:
all_files

In [ ]:
params = read_config_file(params_f)
params

In [ ]:
# cfg = read_config_file(cfg_f)
# cfg

# params["prefix"] = cfg["prefix"]
# params["outdir"] = cfg["outdir"]

params["prefix"] = prefix
params["outdir"] = cfg_outdir

In [ ]:
%load_ext autoreload
%autoreload 2
from src.files import create_single_files


In [ ]:
params_files = create_single_files(params, "clones")
params_files = params_files.set_index('file')

#print(params_files.loc[0, 'file'])

params_files.iloc[0]

In [ ]:
curr = params_files.copy()
print('len curr', curr.shape)

inds_to_drop = []
for ex_params in exclude:
    print(ex_params)
    for k in ex_params:
        print('k', k)
        print('ex_params', ex_params[k])
        curr = curr[curr[k] == ex_params[k]]
        print('len curr', curr.shape)
    inds_to_drop += (list(curr.index))
    
params_files = params_files.copy().loc[(~params_files.index.isin(inds_to_drop))]
print('len params_files', params_files.shape)

In [ ]:
params_files.reset_index()

In [ ]:
#names = names.split(" ")
all_methods = []

all_methods_df = pd.read_csv(all_files[0], sep='\t').set_index("ID")

curr_out = (join(outdir, all_files[0]))#.replace("/", "_")
all_outs = [curr_out]
all_methods_df[curr_out] = all_methods_df["lineage"]
all_methods_df = all_methods_df[["condition", "donor", curr_out] ]

for ind, i in enumerate(all_files[1:]):
    print(i)
    curr_out = (join(outdir, i))#.replace("/", "_") #names[ind])
    curr_df = pd.read_csv(i, sep='\t').set_index("ID")
    curr_df[curr_out] = curr_df["lineage"]
    print('Number duplicated', curr_df.index.duplicated().sum())

    all_outs.append(curr_out)
    all_methods_df = pd.merge(all_methods_df, curr_df[~(curr_df.duplicated())][[curr_out]], 
                              left_index=True, right_index=True,how='outer')
    all_methods_df = all_methods_df[["condition", "donor"]+all_outs]
   # print('all_methods_df')
  #  print(all_methods_df.head())
    #all_methods.append(curr_df) #[curr_out] = curr_df

print('Number duplicated')
print(all_methods_df.duplicated().sum())
all_methods_df = all_methods_df[~(all_methods_df.duplicated())]
print(all_methods_df.duplicated().sum())


## Create cell-pair vector
keys: 
- 0: Not same clone 
- 1: In same clone
- -1: Both are NA
- -2: One is NA


In [ ]:
def calc_cell_pairs(cell_ser, clones_series):
    curr_id = cell_ser.name
    print('curr_id', curr_id)
    print('clones_series', clones_series.head())
    curr_clone = clones_series.loc[curr_id]
    print('curr_clone', curr_clone)
    if np.isnan(curr_clone):
        cell_ser.loc[:] == -1
        cell_ser.loc[clones_series.loc[clones_series.isnull()].index] = -2
    else:
        cell_ser.loc[:] = (curr_clone == clones_series.loc[cell_ser.index])
        cell_ser.loc[clones_series.loc[clones_series.isnull()].index] = -1
    return cell_ser



In [ ]:
donor_all_methods_pairs_df = {}
for d, curr_methods_df in all_methods_df.groupby("donor"):
    print('donor', d)
    donor_all_methods_pairs_df[d] = {}
    # d = 0
    # curr_methods_df = all_methods_df.loc[all_methods_df["donor"]==d]
    #curr_donor_all_methods_pairs_df = {}

    for i in curr_methods_df.drop(["condition", "donor"], axis=1).columns:
        print('i', i)
        cell_pairs_df = pd.DataFrame(index=curr_methods_df.index, columns=curr_methods_df.index)
        donor_all_methods_pairs_df[d][i] = cell_pairs_df.apply(calc_cell_pairs, 
                                                            args=(curr_methods_df[i],), axis=1)



In [ ]:
def calc_meth_overlap(meth_a, meth_b, use_na=False):
    n_T_T = ((meth_a == True) & (meth_b==True)).values.sum()
    n_F_F = ((meth_a == False) & (meth_b==False)).values.sum()
    n_F_T = ((meth_a == False) & (meth_b==True)).values.sum()
    n_T_F = ((meth_a == True) & (meth_b==False)).values.sum()
    
    n_nas_mone = ((meth_a == -1) & (meth_b==-1)).values.sum()
    n_nas_mtwo = ((meth_a == -2) & (meth_b==-2)).values.sum()
    
    n_nas_both = ((meth_a < 0) & (meth_b<0)).values.sum()
    n_nas_mis = (((meth_a < 0) & (meth_b >= 0)) | ((meth_a >= 0) & (meth_b < 0))).values.sum()
    
    
    n_together = n_T_T + n_F_F + n_F_T + n_T_F
    
    #out_meth = meth_a == meth_b
    
    if not use_na:
        meth_a = meth_a.drop(meth_a<0)
        meth_b = meth_b.drop(meth_b<0)    
        
        inds = set(meth_a.index).intersection(set(meth_b.index))
        meth_a = meth_a.loc[inds]
        meth_b = meth_b.loc[inds]
    #jaccard(meth_a, meth_b)
    n_T_T_norm = n_T_T/(n_T_T+n_T_F+n_F_T) #n_together
    n_T_F_norm = n_T_F/n_together
    n_F_T_norm = n_F_T/n_together
    n_F_F_norm = n_F_F/n_together
    return {"n_T_T":n_T_T, "n_F_F":n_F_F,
            "n_F_T":n_F_T, "n_T_F":n_T_F, 
            
            "n_T_T_norm":n_T_T_norm, "n_F_F_norm":n_F_F_norm,
            "n_F_T_norm":n_F_T_norm, "n_T_F_norm":n_T_F_norm, 
            
            "n_nas_mone":n_nas_mone, "n_nas_mtwo":n_nas_mtwo, 
            "n_nas_both":n_nas_both, "n_nas_mis":n_nas_mis,
            "n_together":n_together, 
            "n_agree": n_T_T  +n_F_F
           }

In [ ]:
def get_title(fname, params_files):
    print('fname', fname)
#     pre_ind = fname.find('/')
#     print('pre_ind')
#     print(pre_ind)
#     prefix = fname[:pre_ind]
#     print('prefix', prefix)
#     curr_file = fname[pre_ind:].replace("/concat", "").replace("/cells_meta.tsv", "")
    curr_file = fname.replace("/concat", "").replace("/cells_meta.tsv", "")
    #print('curr_file', curr_file)
    params = params_files.loc[curr_file].iloc[0]
    #print('params', params[["method", "nclonelist", "resolution"]])
    if params['method'] == 'knn':
        meth_params = params['resolution']
    if params['method'] == 'vireo':        
        meth_params = params['nclonelist']
    return f"Variants method: {params['variants']}\n clones method: {params['method']}\nClones params: {meth_params}"


In [ ]:
meth_df = pd.DataFrame(columns=["donor", "m1", "m2", "n_agree", "n_T_T", "n_F_F", "n_F_T", "n_T_F", 
                                "n_T_T_norm", "n_F_F_norm", 
                                "n_F_T_norm", "n_T_F_norm", "n_nas_mone", "n_nas_mtwo", 
                                "n_nas_both", "n_nas_mis", "n_together"])

for d, curr_methods_df in all_methods_df.groupby("donor"):
    print('donor', d)
    curr_d_methods = list(donor_all_methods_pairs_df[d].keys())
    curr_d_pairs = list(itertools.product(curr_d_methods, repeat=2))
    for curr_pair in curr_d_pairs:
        if curr_pair[0] == curr_pair[1]:
            continue
        print("curr_pair")
        print(curr_pair[0])
        print(curr_pair[1])
        a = donor_all_methods_pairs_df[d][curr_pair[0]]
        b = donor_all_methods_pairs_df[d][curr_pair[1]]
        curr_out = calc_meth_overlap(a,b)
        curr_out["donor"] = d
        curr_out["m1"] = get_title(curr_pair[0], params_files) #curr_pair[0]
        curr_out["m2"] = get_title(curr_pair[1], params_files) #curr_pair[1]
        meth_df = meth_df.append(pd.DataFrame(curr_out, index= [f"d{d}_m1{curr_pair[0]}_m2{curr_pair[1]}"]))
        #meth_df.loc[curr_pair]
meth_df["n_agree_norm"] = meth_df["n_agree"]/meth_df["n_together"]

## Average over donors

In [ ]:
# meth_df = pd.DataFrame(columns=["donor", "m1", "m2", "n_agree", "n_T_T", "n_F_F", "n_F_T", "n_T_F", 
#                                 "n_T_T_norm", "n_F_F_norm", 
#                                 "n_F_T_norm", "n_T_F_norm", "n_nas_mone", "n_nas_mtwo", 
#                                 "n_nas_both", "n_nas_mis", "n_together"])

# for d, curr_methods_df in all_methods_df.groupby("donor"):
#     print('donor', d)
#     curr_d_methods = list(donor_all_methods_pairs_df[d].keys())
#     curr_d_pairs = list(itertools.product(curr_d_methods, repeat=2))
#     for curr_pair in curr_d_pairs:
#         if curr_pair[0] == curr_pair[1]:
#             continue
#         print("curr_pair")
#         print(curr_pair[0])
#         print(curr_pair[1])
#         a = donor_all_methods_pairs_df[d][curr_pair[0]]
#         b = donor_all_methods_pairs_df[d][curr_pair[1]]
#         curr_out = calc_meth_overlap(a,b)
#         curr_out["donor"] = d
#         curr_out["m1"] = get_title(curr_pair[0], params_files) #curr_pair[0]
#         curr_out["m2"] = get_title(curr_pair[1], params_files) #curr_pair[1]
#         meth_df = meth_df.append(pd.DataFrame(curr_out, index= [f"d{d}_m1{curr_pair[0]}_m2{curr_pair[1]}"]))
#         #meth_df.loc[curr_pair]
# meth_df["n_agree_norm"] = meth_df["n_agree"]/meth_df["n_together"]

## Correlation Heatmaps for different metrics

In [ ]:
sns.set(font_scale=1.2)
def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')

    metric = args[0] #kwargs.pop(args[0])
    print('metric', metric)
    d = data.pivot(index="m1", columns="m2", values=[args[0]])
    #d = data.pivot(index=args[1], columns=args[0], values=args[2])
    sns.heatmap(d, **kwargs)
    return

def donors_heatmap(df, metric, title=""): 
    df[metric] = df[metric].astype('float')
    g = sns.FacetGrid(df, col="donor", col_wrap=2, height=8, aspect=1)
    g.map_dataframe(draw_heatmap, metric, cbar=True, square=True)
    g.figure.suptitle(title)#"Methods comparison: #pairs in shared clusters clusters")
    plt.tight_layout()
    return

def donors_agg_heatmap(df, metric, title="", to_log=False):
    df = df.copy()
    if to_log:
        df[metric] = np.log10(df[metric]+1)
    df_agg = df.groupby(['m1','m2']).mean().reset_index()
    f = plt.figure(figsize=(8,8), dpi=300)
    draw_heatmap(metric, data=df_agg)
    plt.title(title)#"Methods comparison: #pairs in shared clusters clusters")
    plt.tight_layout()
    return

In [ ]:
df_agg = meth_df.groupby(['m1','m2']).mean()
df_agg.reset_index()

In [ ]:
donors_heatmap(meth_df, metric="n_agree", 
               title="Methods comparison: #pairs with agreed clusters (T or F)")
plt.savefig(join(outdir, "methods_nAgree.png"))

In [ ]:
donors_heatmap(meth_df, metric="n_T_T", 
               title="Methods comparison: #pairs in same clusters (T-T)")
plt.savefig(join(outdir, "methods_nTT.png"))
# for d, df in meth_df.groupby("donor"):
#     f=plt.figure()
#     sns.heatmap(df.pivot(index="m1", columns="m2", values=["n_T_T"]).astype('float'))
#     plt.title(f"Methods comparison: #pairs in shared clusters clusters\n donor {d}")

In [ ]:
donors_heatmap(meth_df, metric="n_agree_norm", 
               title="Methods comparison: Normed #pairs with agreed clusters (T or F)")
plt.savefig(join(outdir, "methods_nAgreeNorm.png"))

# for d, df in meth_df.groupby("donor"):
#     f=plt.figure()
#     sns.heatmap(df.pivot(index="m1", columns="m2", values=["n_agree_norm"]).astype('float'))
#     plt.title(f"Methods comparison: Normed #pairs with agreed clusters (T or F)\n donor {d}")

In [ ]:
donors_heatmap(meth_df, metric="n_T_T_norm", title="Methods comparison: Normed #pairs in agreed clusters (T)")
plt.savefig(join(outdir, "methods_nTTNorm.png"))


In [ ]:
meth_df.to_csv(join(outdir, "methods_comparisons.csv"))

params_files.to_csv(join(outdir, "params_files.csv"))

## Donors aggregate

In [ ]:
donors_agg_heatmap(meth_df, metric="n_agree", to_log=True,
               title="Methods comparison: #pairs with concordant clone assignment (T-T or F-F) \nlog10 counts")
plt.savefig(join(outdir, "methods_nAgree_agg.png"))

donors_agg_heatmap(meth_df, metric="n_agree_norm", title="Methods comparison: Normed #pairs in concordance (T-T or F-F)")
plt.savefig(join(outdir, "methods_nAgreeNorm_agg.png"))

donors_agg_heatmap(meth_df, metric="n_T_T", to_log=True,
               title="Methods comparison: #pairs in same clones across methods (T-T) \nlog10 counts")
plt.savefig(join(outdir, "methods_n_T_T_agg.png"))

donors_agg_heatmap(meth_df, metric="n_T_T_norm", title="Methods comparison: Normed #pairs in agreed clones (T-T)")
plt.savefig(join(outdir, "methods_nTTNorm_agg.png"))


